In [1]:
!pip install pymorphy2
!pip install pyldavis

     |████████████████████████████████| 51kB 1.6MB/s 
     |████████████████████████████████| 7.1MB 7.0MB/s 
     |████████████████████████████████| 1.6MB 2.7MB/s 
     |████████████████████████████████| 552kB 22.4MB/s 
  Created wheel for pyldavis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=fcc96ed1e6d0c36e809cba39922136047e4888abe1dfb03936fdf2a55891fbee
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=18ffa959b0dcb4e7bf2ae51ac5b868c50e34c8be20279b3305275088e74168ee
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyldavis funcy


In [3]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from pymorphy2 import MorphAnalyzer
import re
import pandas as pd
from nltk import ngrams
import gensim.corpora as corpora
import gensim
from gensim.models import CoherenceModel
from gensim.models.ldamulticore import LdaMulticore
from tqdm import tqdm_notebook
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df = pd.read_csv("cleanAppStore.csv")
df = df[["Review"]]

In [13]:
push_df = df[df.Review.str.contains("push") | df.Review.str.contains("пуш")]
dark_df = df[df.Review.str.contains("темная")]
df_unite = pd.concat([push_df, dark_df])
texts = df_unite.Review.tolist()
texts = [text.replace("ё","e") for text in texts]

In [14]:
mystop = [""]

In [15]:
morph = MorphAnalyzer()
stop = set(stopwords.words('russian')+mystop)

In [16]:
def lemmatize_texts(texts):
    tokenized_text = []
    for text in tqdm_notebook(texts):
        prog = re.compile(r'[а-яa-z]+')
        tokens = prog.findall(str(text).lower())
        res=[morph.parse(token)[0].normal_form for token in tokens if morph.parse(token)[0].normal_form not in stop and len(morph.parse(token)[0].normal_form)>2]
        tokenized_text.append(res)
    return tokenized_text

In [18]:
lemmatized_texts = lemmatize_texts(texts)
lemmatized_texts = [[word for word in text if word not in ["банка"]] for text in lemmatized_texts]
df_unite["lemmatized_texts"] = lemmatized_texts

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
bigrams = []
trigrams = []

for text in df_unite.lemmatized_texts.tolist():
  bigrams.append(["_".join(gram) for gram in ngrams(text, 2)])

for text in df_unite.lemmatized_texts.tolist():
  trigrams.append(["_".join(gram) for gram in ngrams(text, 3)])

df_unite["bigrams"] = bigrams
df_unite["trigrams"] = trigrams


all_grams = []
for row in df_unite.iterrows():
  all_grams.append((row[1]["lemmatized_texts"]+row[1]["bigrams"]+row[1]["trigrams"]))


flat_list = [word for block in all_grams for word in block]
dict_ = {x: count for x, count in Counter(flat_list).items() if count >1}


for i in range(len(all_grams)):
  all_grams[i] = [elem for elem in all_grams[i] if elem in list(dict_.keys())]

'\nfor i in range(len(all_grams)):\n  #print(i)\n  all_grams[i] = [elem for elem in all_grams[i] if elem in list(dict_.keys())]\n'

In [20]:
id2word = corpora.Dictionary(lemmatized_texts)
corpus = [id2word.doc2bow(text) for text in lemmatized_texts]

In [22]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    coherence_values = []

    for num_topics in range(start, limit, step):

        print(num_topics)

        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

        coherencemodel = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return coherence_values

In [ ]:
%%time
coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=lemmatized_texts, start=2, limit=100, step=1)

In [ ]:
limit=200; start=2; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.axvline(x = 73, ymin = 0, ymax = 1, color ='red', linestyle="--") 
#plt.legend(("coherence_values"), loc='best')
plt.show()

In [23]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=73, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [24]:
import pyLDAvis
from pyLDAvis import gensim as sim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
32    -0.232808 -0.065016       1        1  9.800295
62    -0.268882  0.019284       2        1  5.248409
51    -0.247827 -0.054168       3        1  4.783188
50    -0.249460 -0.002747       4        1  4.464586
44    -0.294965  0.170958       5        1  3.726341
...         ...       ...     ...      ...       ...
28     0.105008  0.014395      69        1  0.030698
69     0.105008  0.014395      70        1  0.030634
38     0.105008  0.014395      71        1  0.030632
9      0.105008  0.014395      72        1  0.030472
25     0.105008  0.014395      73        1  0.030472

[73 rows x 5 columns], topic_info=            Term        Freq       Total Category  logprob  loglift
41          тема   46.000000   46.000000  Default   30.000  30.0000
9     приложение  158.000000  158.000000  Default   29.000  29.0000
10   уведомление  193.000000  193.000000  Default   28.000  28.0000
75           смс  126.000000  126.000000  Default   27.000  27.0000
0           push   88.000000   88.000000  Default   26.000  26.0000
..           ...         ...         ...      ...      ...      ...
566     изменить    0.001096    1.776429  Topic73   -7.453   0.7057
564    задолбать    0.001096    1.351731  Topic73   -7.453   0.9789
591          мол    0.001096    1.311233  Topic73   -7.453   1.0093
563       данный    0.001096    3.734915  Topic73   -7.453  -0.0374
540      решение    0.001096    2.825130  Topic73   -7.453   0.2417

[3196 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
398       6  0.633285     apple
1378     23  0.714442      card
1171     25  0.703398      face
1379     23  0.714442      gold
26        3  0.951413       ios
...     ...       ...       ...
178      44  0.011256       это
178      45  0.011256       это
81        4  0.852589  являться
1720     42  0.799101     яркий
1579     30  0.738263      ясно

[2025 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[33, 63, 52, 51, 45, 30, 40, 42, 6, 34, 22, 18, 23, 61, 48, 5, 47, 35, 3, 15, 4, 73, 27, 50, 7, 66, 32, 24, 14, 56, 55, 57, 65, 60, 54, 8, 69, 11, 58, 20, 1, 28, 41, 12, 38, 72, 59, 25, 2, 62, 46, 37, 71, 44, 53, 64, 17, 16, 68, 43, 19, 67, 36, 49, 21, 9, 13, 31, 29, 70, 39, 10, 26])

In [ ]:
pyLDAvis.save_html(vis, 'lda.html')